In [11]:
import os

import obspy
from obspy.core.utcdatetime import UTCDateTime

import matplotlib.pyplot as plt
%matplotlib inline

from scipy import signal
from scipy.interpolate import griddata
import numpy as np
import pandas as pd

from time import sleep
from IPython.display import clear_output, display

import ray
import multiprocessing

n_cpus = multiprocessing.cpu_count()

In [12]:
def lag_finder(y1, y2, channel=0):
    if  type(y1) != obspy.core.stream.Trace:
        y1 = y1[channel]
        y2 = y2[channel]
    
    n = len(y1)
    sr = int(y1.stats.sampling_rate)
    corr = signal.correlate(y2, y1, mode='same') / np.sqrt(signal.correlate(y1, y1, mode='same')[int(n/2)] * signal.correlate(y2, y2, mode='same')[int(n/2)])

    delay_arr = np.linspace(-0.5*n/sr, 0.5*n/sr, n)
    delay = delay_arr[np.argmax(corr)]
    return delay, corr

def get_coordinates(st, inv_ff):
    coordinates = inv_ff.get_coordinates(st[0].stats.network +'.' + st[0].stats.station + '..' + st[0].stats.channel)
    lat = coordinates['latitude']
    lon = coordinates['longitude']
    return lat, lon

def great_circle_dist(lat1, lat2, lon1, lon2):
    lat1, lat2, lon1, lon2 = np.deg2rad([lat1, lat2, lon1, lon2])
    d_sigma = np.arccos(np.sin(lat1)*np.sin(lat2)+np.cos(lat1)*np.cos(lat2)*np.cos(lon2-lon1))
    return 6371*10**3*d_sigma


In [13]:
@ray.remote
def high_processing(station, starttime, endtime, st0):
    st = obspy.read(path+'nodes/FF_Vel/' + station, starttime=starttime, endtime=endtime).detrend()
    st.taper(0.1)
    st.filter('bandpass', freqmin=freqmin, freqmax=freqmax);
    if station == '2019_134_R4_23VEL.mseed':
        print(station, st0[0].stats.starttime, st[0].stats.starttime)
    delays_Z, max_corrs_Z, delays_N, max_corrs_N, delays_E, max_corrs_E, timestamps, stations, lats, lons = zip(*[processing(st0_win, st_win) for st0_win, st_win in zip(st0.slide(window_length=window_length, step=step), 
                               st.slide(window_length=window_length, step=step))])
    
    cross_correlations = pd.DataFrame({'timestamp':timestamps, 
                                    'station':stations, 
                                    'delay_Z':delays_Z, 
                                    'delay_N':delays_N, 
                                    'delay_E':delays_E,
                                    'corr_Z':max_corrs_Z,
                                    'corr_N':max_corrs_N,
                                    'corr_E':max_corrs_E,
                                    'lat':lats, 
                                    'lon':lons})
    
    return cross_correlations


def processing(st0_win, st_win):
    coordinates = get_coordinates(st_win, inv_ff)
    (delay_Z, corr_Z), (delay_N, corr_N), (delay_E, corr_E) = [lag_finder(st0_win, st_win, channel=i) for i in range(0, len(st_win))]
    timestamp = st_win[0].stats.starttime._get_datetime()
    station_name = st_win[0].stats.network +'.' + st_win[0].stats.station
    lat = coordinates[0]
    lon = coordinates[1]
    
    max_corrs_Z = np.abs(corr_Z.max())
    max_corrs_N = np.abs(corr_N.max())
    max_corrs_E = np.abs(corr_E.max())
    
    return delay_Z, max_corrs_Z, delay_N, max_corrs_N, delay_E, max_corrs_E, timestamp, station_name, lat, lon



In [14]:
@ray.remote
def high_processing_a2a(station, starttime, endtime, st0):
    st = obspy.read(path+'nodes/FF_Vel/' + station, starttime=starttime, endtime=endtime).detrend()
    st.taper(0.1)
    st.filter('bandpass', freqmin=freqmin, freqmax=freqmax);
    
    delays_Z, max_corrs_Z, delays_N, max_corrs_N, delays_E, max_corrs_E, timestamps, stations, lats, lons = zip(*[processing_a2a(st0_win, st_win) for st0_win, st_win in zip(st0.slide(window_length=window_length, step=step), 
                               st.slide(window_length=window_length, step=step))])
    cross_correlations = pd.DataFrame({'timestamp':timestamps, 
                                    'station':stations, 
                                    'delay_Z':delays_Z, 
                                    'delay_N':delays_N, 
                                    'delay_E':delays_E,
                                    'corr_Z':max_corrs_Z,
                                    'corr_N':max_corrs_N,
                                    'corr_E':max_corrs_E,
                                    'lat':lats, 
                                    'lon':lons})
    
    return cross_correlations

def processing_a2a(st0_win, st_win):
    coordinates = get_coordinates(st_win, inv_ff)
    (delay_Z, corr_Z), (delay_N, corr_N), (delay_E, corr_E) = [lag_finder(st0_win, st_win, channel=i) for i in range(0, len(st_win))]
    timestamp = st_win[0].stats.starttime._get_datetime()
    station_name = (st0_win[0].stats.network +'.' + st0_win[0].stats.station) + '_' + (st_win[0].stats.network +'.' + st_win[0].stats.station)
    lat = coordinates[0]
    lon = coordinates[1]
    
    max_corrs_Z = np.abs(corr_Z.max())
    max_corrs_N = np.abs(corr_N.max())
    max_corrs_E = np.abs(corr_E.max())
    
    return delay_Z, max_corrs_Z, delay_N, max_corrs_N, delay_E, max_corrs_E, timestamp, station_name, lat, lon


In [15]:
@ray.remote
def high_processing_hyperions(st0, st):
    if st.stats != st0.stats: 
        delays, max_corrs, timestamps, stations_from, stations_to, lats, lons = zip(*[processing_hyperions(st0_win, st_win) for st0_win, st_win in zip(st0.slide(window_length=window_length, step=step), 
                                   st.slide(window_length=window_length, step=step))])

        cross_correlations = pd.DataFrame({'timestamp':timestamps, 
                                        'station_from':stations_from,
                                        'station_to':stations_to,
                                        'delay':delays, 
                                        'corr':max_corrs,
                                        'lat':lats, 
                                        'lon':lons})

        return cross_correlations
    else:
        pass

def processing_hyperions(st0_win, st_win):
    coordinates = get_coordinates_hyperions(st0_win, inv_hyp)
    (delay, corr) = lag_finder(st0_win, st_win)
    timestamp = st_win.stats.starttime._get_datetime()
    station_from = st0_win.stats.station + '.' + st0_win.stats.location
    station_to   = st_win.stats.station + '.'  + st_win.stats.location
    lat = coordinates[0]
    lon = coordinates[1]
    max_corrs = np.abs(corr.max())
    return delay, max_corrs, timestamp, station_from, station_to, lat, lon

def get_coordinates_hyperions(st, inv_hyp):
    coordinates = inv_hyp.get_coordinates(st.stats.network +'.' + 
                                          st.stats.station + '.' + 
                                          st.stats.location + '.' + 
                                          st.stats.channel)
    lat = coordinates['latitude']
    lon = coordinates['longitude']
    return lat, lon

In [16]:
def calc_t_est(delay_matrix):
    n = delay_matrix.shape[0]
    a_arr = np.zeros((n,1))
    b_arr = np.zeros((n,1))
    
    for i in range(1,5):
        a=0.0
        for j in range(0,i-1):
            a+=delay_matrix[j,i-1]
            a_arr[i-1]=a
    
    for i in range(1,5):
        b = 0.0
        for j in range((i-1)+1, n):
            b+=delay_matrix[i-1,j]
            b_arr[i-1]=b

    return (1/n)*(-a_arr+b_arr)

def plot_maps(x, y, z, v, cmap, metlift_db, npoints=100, quiver=True):
        xi = np.linspace(x.min(), x.max(), npoints)
        yi = np.linspace(y.min(), y.max(), npoints)
        zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='cubic', rescale=True)
        
        fig = plt.figure(figsize=(7,4))
        ax1 = fig.add_subplot(111)
        ax1.contourf(xi,yi,zi,10,cmap=cmap, alpha=0.5, vmin=-v, vmax=v)

        c = ax1.scatter(x, y, c=z, vmin=-v, vmax=v, cmap=cmap)
        ax1.set_xlim(x.min()-0.00005, x.max()+0.0003)
        ax1.set_ylim(y.min()-0.00005, y.max()+0.00005)
        plt.suptitle(name)
        plt.colorbar(c)

        at_time = metlift_db[metlift_db['timestamp'] == name]
        wind_dir = at_time['Wind Grad'].values[0]
        ax2 = fig.add_subplot(111, projection='polar', label='polar')
        box = ax2.get_position()
        ax2.set_position([0.505, 0.33, box.width * 0.45 , box.height * 0.45])

        ax2.patch.set_alpha(0.5)
        ax2.plot((0, np.deg2rad(wind_dir)), (0, at_time['Wind 1m MW'].values[0]))
        
        ##Direction estimation
        gy, gx = np.gradient(zi)
        wind_abs = np.sqrt(gx**2 + gy**2)
        wind_dir_trig = np.arctan2(gx/wind_abs, gy/wind_abs) 
        
        estimated_direction = wind_dir_trig[~np.isnan(wind_dir_trig)].mean()
        estimated_speed = wind_abs[~np.isnan(wind_abs)].mean()

        ax2.plot((0, estimated_direction), (0, estimated_speed*1e5))
        print(np.rad2deg(estimated_direction)%360, wind_dir)
        
        ax2.set_theta_zero_location("N")
        ax2.set_theta_direction(-1)
        ax2.set_rmax(metlift_db['Wind 1m MW'].max())
        if quiver:
            ax1.quiver(xi[1:-1:10], yi[1:-1:10], gx[1:-1:10,1:-1:10], gy[1:-1:10, 1:-1:10], angles='xy', alpha=0.25)

## Fair Fields

In [17]:
path = 'Ground_coupling/'
inv_ff = obspy.read_inventory(path+'nodes/nodes_metadata.xml')
st_list = os.listdir(path+'nodes/FF_Vel/')

In [18]:
# Configure parameters
freqmin = 10
freqmax = 200
window_length = 1.0
step = 0.5

# Set step time
step_time = 10*60


### Cross-correlation all to `R1.1`

In [19]:
%%time
####FAIR FIELDS####

# Configure time intervals
starttime = UTCDateTime('2019-05-14 12:45:00')
starttimes = np.asarray([starttime + i for i in np.arange(0, 100*60, step_time)])
endtimes = np.asarray(starttimes)+(step_time)

mid_results = []
final_results = []

st0_full = obspy.read(path+'nodes/FF_Vel/2019_134_R1_1VEL.mseed')
lat0, lon0 = get_coordinates(st0_full, inv_ff)

# Initialize RAY
if ray.is_initialized():
    ray.shutdown() 
ray.init(redis_max_memory=int(2.25e+11), object_store_memory=int(1e+11))

for starttime, endtime in zip(starttimes, endtimes):
    print(starttime)

    # Load seismic
    st0 = obspy.read(path+'nodes/FF_Vel/2019_134_R1_1VEL.mseed', starttime=starttime, endtime=endtime).detrend();
    st0.taper(0.1)
    st0.filter('bandpass', freqmin=freqmin, freqmax=freqmax);
    
    # Compute
    results_id = [high_processing.remote(station, starttime, endtime, st0) for station in st_list[:]]
    mid_results.append(results_id)
final_results.append([ray.get(result) for result in mid_results])

# Kill RAY
ray.shutdown()

# Extract cross-correlations
cross_correlations = pd.concat([pd.concat(item) for item_big in final_results for item in item_big])
cross_correlations['timestamp_round_min'] = cross_correlations['timestamp'].dt.round('min')
cross_correlations['timestamp_round_sec'] = cross_correlations['timestamp'].dt.round('s')
cross_correlations.to_pickle('cross_correlations.pkl')


2019-07-16 14:35:09,629	WARNING worker.py:1337 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-07-16 14:35:09,633	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-16_14-35-09_630252_22186/logs.
2019-07-16 14:35:09,822	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:46485 to respond...
2019-07-16 14:35:10,006	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:13432 to respond...
2019-07-16 14:35:10,009	INFO services.py:806 -- Starting Redis shard with 225.0 GB max memory.
2019-07-16 14:35:10,183	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-16_14-35-09_630252_22186/logs.
2019-07-16 14:35:10,186	WARNING services.py:1323 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 9793597440 bytes available. Thi

2019-05-14T12:45:00.000000Z


2019-07-16 14:35:11,796	WARNING worker.py:343 -- WARNING: Falling back to serializing objects of type <class 'obspy.core.trace.Stats'> by using pickle. This may be inefficient.


2019-05-14T12:55:00.000000Z
(pid=31253) 2019_134_R4_23VEL.mseed 2019-05-14T12:45:00.000000Z 2019-05-14T12:45:00.000000Z
2019-05-14T13:05:00.000000Z
2019-05-14T13:15:00.000000Z
2019-05-14T13:25:00.000000Z
2019-05-14T13:35:00.000000Z
2019-05-14T13:45:00.000000Z
2019-05-14T13:55:00.000000Z
2019-05-14T14:05:00.000000Z
2019-05-14T14:15:00.000000Z
(pid=31278) 2019_134_R4_23VEL.mseed 2019-05-14T12:55:00.000000Z 2019-05-14T12:55:00.000000Z


Exception in thread ray_import_thread:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/redis/connection.py", line 185, in _read_from_socket
    raise socket.error(SERVER_CLOSED_CONNECTION_ERROR)
OSError: Connection closed by server.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/import_thread.py", line 69, in _run
    msg = import_pubsub_client.get_message()
  File "/opt/anaconda3/lib/python3.7/site-packages/redis/client.py", line 3135, in get_message
    response = self.parse_response(block=False, timeout=timeout)
  File "/opt/anaconda3/lib/python3.7/site-packages/redis/client.py", line 3036, in parse_response
    return self._e

KeyboardInterrupt: 

### Cross-correlation all to `All lto All`

In [10]:
%%time

# Set step time
step_time = 30*60

# Configure time intervals
starttime = UTCDateTime('2019-05-14 12:45:00')
starttimes = np.asarray([starttime + i for i in np.arange(0, 90*60, step_time)])
endtimes = np.asarray(starttimes)+(step_time)

mid_results = []
final_results = []
cc_a2a_list = []

# Configure parameters
window_length = 0.4
step = 0.2

for starttime, endtime in zip(starttimes, endtimes):
    print(starttime)
    
    # Initialize RAY
    if ray.is_initialized():
        ray.shutdown() 
    ray.init(redis_max_memory=int(2.25e+11), object_store_memory=int(1e+11))


    for station in st_list[:]:
        # Load seismic
        st0 = obspy.read(path+'nodes/FF_Vel/' + station, starttime=starttime, endtime=endtime).detrend();
        st0.taper(0.1);
        st0.filter('bandpass', freqmin=freqmin, freqmax=freqmax);

        # Compute
        results_id = [high_processing_a2a.remote(station, starttime, endtime, st0) for station in st_list[:]]
        mid_results.append(results_id)
    final_results.append([ray.get(result) for result in mid_results])
    cross_correlations_a2a = pd.concat([pd.concat(item) for item_big in final_results for item in item_big])
    cc_a2a_list.append(cross_correlations_a2a)
    
    # Kill RAY
    ray.shutdown()

# Extract cross-correlations

cross_correlations_a2a = pd.concat(cc_a2a_list)
cross_correlations_a2a['timestamp_round_min'] = cross_correlations_hyperions['timestamp'].dt.round('min')
cross_correlations_a2a['timestamp_round_sec'] = cross_correlations_hyperions['timestamp'].dt.round('s')
cross_correlations_a2a.to_pickle('cross_correlations_a2a.pkl')




2019-07-16 07:28:19,024	WARNING worker.py:1337 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-07-16 07:28:19,032	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-16_07-28-19_026951_22186/logs.
2019-07-16 07:28:19,145	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:45192 to respond...


2019-05-14T12:45:00.000000Z


2019-07-16 07:28:19,258	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:35398 to respond...
2019-07-16 07:28:19,260	INFO services.py:806 -- Starting Redis shard with 225.0 GB max memory.
2019-07-16 07:28:19,283	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-16_07-28-19_026951_22186/logs.
2019-07-16 07:28:19,286	INFO services.py:1446 -- Starting the Plasma object store with 100.0 GB memory using /dev/shm.
2019-07-16 07:28:22,635	WARNING worker.py:343 -- WARNING: Falling back to serializing objects of type <class 'obspy.core.trace.Stats'> by using pickle. This may be inefficient.
2019-07-16 07:29:55,076	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 07:30:05,085	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstru

2019-07-16 07:36:58,174	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 07:37:08,291	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 07:37:18,378	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 07:37:28,478	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 07:37:38,578	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 07:37:48,718	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is

2019-07-16 07:44:42,944	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 07:44:53,036	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 07:45:03,148	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 07:45:13,206	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 07:45:23,301	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 07:45:33,393	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is

...

2019-07-16 10:34:35,537	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:34:45,580	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:34:55,616	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:35:05,651	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:35:15,691	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:35:25,741	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is

2019-07-16 10:42:17,438	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:42:27,481	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:42:37,526	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:42:47,559	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:42:57,600	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:43:07,647	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is

2019-07-16 10:49:59,375	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:50:09,417	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:50:19,460	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:50:29,509	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:50:39,549	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:50:49,586	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is

2019-07-16 10:57:41,314	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:57:51,355	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:58:01,389	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:58:11,434	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:58:21,476	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is a driver task and so the object created by ray.put could not be reconstructed.
2019-07-16 10:58:31,511	ERROR worker.py:1672 -- The task with ID f3d6ba5c6e29b0231402e20f13a2923c is

KeyboardInterrupt: 

## Hyperions

In [ ]:
inv_hyp = obspy.read_inventory(path+'hyperions/hyperion_metadata.xml')
# hyp = obspy.read(path+'hyperions/hyperions_VEL.mseed', starttime=UTCDateTime('2019-05-14 13:00:00'), endtime=UTCDateTime('2019-05-14 13:10:00'))
hyp = obspy.read(path+'hyperions/hyperions_VEL.mseed').detrend()

In [ ]:
####HYPERIONS####
%%time

# Set step time
step_time = 10*60

# Configure time intervals
starttime = UTCDateTime('2019-05-14 12:55:00')
starttimes = np.asarray([starttime + i for i in np.arange(0, 90*60, step_time)])
endtimes = np.asarray(starttimes)+(step_time)

mid_results = []
final_results = []

# Configure parameters
window_length = 0.4
step = 0.2
    
# Initialize RAY
if ray.is_initialized():
    ray.shutdown() 
ray.init(redis_max_memory=int(2.25e+11), object_store_memory=int(1e+11))
for starttime, endtime in zip(starttimes, endtimes):
    print(starttime)
    hyp_trimed = hyp.copy().trim(starttime=starttime, endtime=endtime).detrend()
    hyp_trimed.taper(0.1).filter('bandpass', freqmin=freqmin, freqmax=freqmax)
    
    for st0 in [hyp_trimed[i] for i in range(0,len(hyp_trimed))]:
        # Compute
        results_id = [high_processing_hyperions.remote(st0, st) for st in [hyp_trimed[i] for i in range(0,len(hyp_trimed))]]
        mid_results.append(results_id)
final_results.append([ray.get(result) for result in mid_results])

# Kill RAY
ray.shutdown()

# Extract cross-correlations

cross_correlations_hyperions = pd.concat([pd.concat(item) for item_big in final_results for item in item_big])
cross_correlations_hyperions['timestamp_round_min'] = cross_correlations_hyperions['timestamp'].dt.round('min')
cross_correlations_hyperions['timestamp_round_sec'] = cross_correlations_hyperions['timestamp'].dt.round('s')
cross_correlations_hyperions.to_pickle('cross_correlations_hyperions.pkl')


## Meteorology

In [ ]:
metlift_db = pd.read_csv('Ground_coupling/MetLift/01080085_190514000000_-_190515000000.csv')
metlift_db = metlift_db.drop(metlift_db.index[0])
metlift_db = metlift_db.rename(columns = {'Unnamed: 0' : 'timestamp'})
metlift_db['timestamp'] =  pd.to_datetime(metlift_db['timestamp'])

for column in metlift_db.columns:
    if column != 'timestamp':
        metlift_db[column] = pd.to_numeric(metlift_db[column])

metlift_db.to_pickle('MetLift.pkl')

## Fair Fields

#### Mean time (0.30 - 1.30)

In [ ]:
cmap='seismic'
cross_correlations = pd.read_pickle('cross_correlations.pkl')
cross_correlations = cross_correlations[~cross_correlations['station'].isin(['C1.1', 'C1.2', 'C1.3', 'R1.1'])]
# cross_correlations = cross_correlations[cross_correlations['station'].isin(['R1.1', 'R5.1', 'R5.9', 'R5.17', 'R5.25',
#                                                                            'R6.1', 'R6.2', 'R6.3', 'R6.4', 'R6.5', 'R6.6', 'R6.7', 'R6.8'])]
cross_correlations_test = cross_correlations[cross_correlations.corr_Z > 0.0]
v = 0.03
for name, group in cross_correlations_test.groupby('timestamp_round_min'):
    if name in pd.to_datetime(cross_correlations_test['timestamp_round_min'].unique()[[12]]):
        cross_rounded = group.groupby('station').mean()

        x = cross_rounded.lon
        y = cross_rounded.lat
        z = cross_rounded.delay_Z

        plot_maps(x, y, z, v, cmap, metlift_db, quiver=True)


    

#### Exact time

In [ ]:
cross_correlations = cross_correlations[~cross_correlations['station'].isin(['C1.1', 'C1.2', 'C1.3'])]
cross_correlations_test = cross_correlations[cross_correlations['timestamp'].isin(cross_correlations['timestamp_round_min'].unique())]
cross_correlations_test = cross_correlations_test[cross_correlations_test.corr_Z > 0.0]
v = 0.5
for name, group in cross_correlations_test.groupby('timestamp'):
    if name in pd.to_datetime(cross_correlations_test['timestamp'].unique()[[1, 10, 25]]):
        cross_rounded = group.groupby('station').mean()

        x = cross_rounded.lon
        y = cross_rounded.lat
        z = cross_rounded.delay_Z

        plot_maps(x, y, z, v, cmap, metlift_db)


    

In [ ]:
names = []
delays_Z = []

for name, group in cross_correlations.groupby('timestamp_round_min'):
#     cross_rounded = group.groupby('station').median()
    names.append(name)
    delays_Z.append(np.abs(group['delay_Z'].mean()))

    
wind_speed = metlift_db[metlift_db['timestamp'].isin(names)]['Wind 1m MW'].values
plt.scatter(wind_speed, delays_Z)
plt.ylim(0, 0.004);

# Preview

In [ ]:
starttime_trim = UTCDateTime('2019-05-14 12:58:19.2')
endtime_trim   = starttime_trim+window_length
# st0[0].plot(starttime=starttime_trim, endtime=endtime_trim);

st = obspy.read(path+'nodes/FF_Vel/' + st_list[1], starttime=starttime_trim, endtime=endtime_trim)
st.filter('bandpass', freqmin=freqmin, freqmax=freqmax)
st0 = obspy.read(path+'nodes/FF_Vel/2019_134_R1_1VEL.mseed', starttime=starttime_trim, endtime=endtime_trim)
st0.filter('bandpass', freqmin=freqmin, freqmax=freqmax);


channel = 0

st0_win = st0.copy().trim(starttime=starttime_trim, endtime = endtime_trim).normalize()
st_win  = st.copy().trim(starttime=starttime_trim,  endtime = endtime_trim).normalize()

fig = plt.figure(figsize=(20,5))
delay, corr = lag_finder(st0_win, st_win, channel=channel)

name = st_win[channel].stats.network+'.'+st_win[channel].stats.station

ax1 = fig.add_subplot(311)
ax1.plot(st0_win[channel].times(), st0_win[channel].data**2, label = 'R1.1')
# ax1.plot(st_win[channel].times() , st_win[channel].data**2, alpha=0.5, linestyle='--', label='Not-shifted %s' % name)
# ax1.plot(st_win[channel].times()-delay , st_win[channel].data**2, label='Shifted %s' % name)
# ax1.legend()
ax1.set_title('Waveform %s' % st_win[channel].stats.channel)

ax2 = fig.add_subplot(313)
ax2.plot(corr, label='Correlation function', color='black')
ax2.legend()
print('Delay: %.5f ms; Correlation: %.3f' % (delay*1000, np.abs(corr).max()))

ax3 = fig.add_subplot(312)
for i in range(0,4):
    ax3.plot(hyp[i].times(), hyp[i].data, label = hyp[i].stats.location)
ax3.legend()

In [ ]:
hyp[0].spectrogram()

In [ ]:
st0[0].spectrogram()

In [ ]:
starttime_trim = UTCDateTime('2019-05-14 14:24:59.600')
endtime_trim   = starttime_trim+window_length
# st0[0].plot(starttime=starttime_trim, endtime=endtime_trim);

st = obspy.read(path+'nodes/FF_Vel/' + st_list[1], starttime=starttime_trim, endtime=endtime_trim)
st.filter('bandpass', freqmin=freqmin, freqmax=freqmax)
st0 = obspy.read(path+'nodes/FF_Vel/2019_134_R1_1VEL.mseed', starttime=starttime_trim, endtime=endtime_trim)
st0.filter('bandpass', freqmin=freqmin, freqmax=freqmax);


delay, corr = lag_finder(st0[0], st0[0])
plt.plot(corr)
print(delay)

In [ ]:
hyp = obspy.read(path+'hyperions/hyperions_VEL.mseed', starttime=starttime_trim, endtime=endtime_trim).detrend().normalize()
hyp.taper(0.1)
hyp.filter('bandpass', freqmin=freqmin, freqmax=freqmax);
hyp.plot()

### Fisher 

In [ ]:
cross_correlations = cross_correlations[cross_correlations['corr_Z'] != 1]
cross_correlations['Fisher'] = 0.5* np.log((1+cross_correlations['corr_Z'])/(1-cross_correlations['corr_Z']))
# cross_correlations['Fisher_r'] = (np.exp(2*cross_correlations['Fisher'])-1)/(np.exp(2*cross_correlations['Fisher'])+1)

In [ ]:
cross_correlations = cross_correlations[~cross_correlations['station'].isin(['C1.1', 'C1.2', 'C1.3'])]

v = 0.5
cmap = 'seismic'
for name, group in cross_correlations.groupby('timestamp_round_min'):
    if name in pd.to_datetime(cross_correlations['timestamp_round_min'].unique()[[25]]):
        cross_rounded = group
#         cross_rounded = cross_rounded[cross_rounded['Fisher'] != np.inf]
#         cross_rounded['corr_Z'].hist(bins=100)
#         cross_rounded['Fisher'].hist(bins=100, alpha=0.5)

        npoints = 100
        
        x = cross_rounded.lon
        y = cross_rounded.lat
        z = cross_rounded.delay_Z

        plot_maps(x, y, z, v, cmap, metlift_db)